In [1]:
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
from datetime import datetime

In [2]:
path = os.getcwd()

bulan = {"maret": "03", "april": "04", "mei": "05", "juni": "06", "juli": "07", "agustus": "08", 
         "september": "09", "oktober": "10", "november": "11"}
kolom_diambil = ["nama_kota", "nama_kecamatan","positif"]

## Only for months of 3,  4,  5,  6,  9, 10

In [3]:
#Process Data
df = pd.DataFrame()
for (dirpath, dirnames, filenames) in tqdm(os.walk(path)):
    for filename in filenames:
        if filename.endswith('.csv'):
            temp = re.search('tanggal-(.+?)-2020', filename)
            if temp:
                temp2 = temp.group(1)
                for key in bulan.keys():
                    temp2 = temp2.replace(key, bulan[key])
                df_temp = pd.read_csv(os.sep.join([dirpath, filename]))
                df_temp = df_temp[kolom_diambil]
                df_temp = df_temp.groupby(["nama_kota", "nama_kecamatan"]).sum()
                df_temp["tanggal"] = datetime.strptime("{}-2020".format(temp2), '%d-%m-%Y').date()
                df = pd.concat([df, df_temp], axis=0)

11it [00:07,  1.55it/s]


In [4]:
#Make the dataframe tidy
df = df.loc(axis=0)["JAKARTA TIMUR"].reset_index().sort_values(["nama_kecamatan", "tanggal"]).reset_index(drop=True)

In [5]:
df.tail(3)

,nama_kecamatan,positif,tanggal
1587,PULO GADUNG,1941,2020-10-29
1588,PULO GADUNG,1965,2020-10-30
1589,PULO GADUNG,1978,2020-10-31


## The rest of the months (7 and 11)

### July

In [6]:
#Process Data
df_2 = pd.DataFrame()
for (dirpath, dirnames, filenames) in tqdm(os.walk(path)):
    for dirname in dirnames:
        if dirname.endswith("Juli"):
            path_july = os.sep.join([dirpath, dirname])
            break

for (dirpath, dirnames, filenames) in tqdm(os.walk(path_july)):
    for filename in filenames:
        tanggal = re.search("\((.+?)\)", filename).group(1)
        if filename.endswith('.csv'):
            df_2_temp = pd.read_csv(os.sep.join([dirpath, filename]))
            df_2_temp = df_2_temp[kolom_diambil]
            df_2_temp = df_2_temp.groupby(["nama_kota", "nama_kecamatan"]).sum()
            df_2_temp["tanggal"] = datetime.strptime("{}-07-2020".format(tanggal), '%d-%m-%Y').date()
            df_2 = pd.concat([df_2, df_2_temp], axis=0)

11it [00:00, 999.94it/s]
1it [00:01,  1.25s/it]


In [7]:
#Make the dataframe tidy
df_2 = df_2.loc(axis=0)["JAKARTA TIMUR"].reset_index().sort_values(["nama_kecamatan", "tanggal"]).reset_index(drop=True)

### November

In [8]:
#Process Data
for (dirpath, dirnames, filenames) in tqdm(os.walk(path)):
    for dirname in dirnames:
        if dirname.endswith("November"):
            path_nov = os.sep.join([dirpath, dirname])
            break

df_3 = pd.DataFrame()
for (dirpath, dirnames, filenames) in tqdm(os.walk(path_nov)):
    for filename in filenames:
        if filename.endswith('.csv'):
            temp = filename[-20:-4]
            for key in bulan.keys():
                temp = temp.replace(key, bulan[key])
            df_3_temp = pd.read_csv(os.sep.join([dirpath, filename]))
            df_3_temp = df_3_temp[kolom_diambil]
            df_3_temp = df_3_temp.groupby(["nama_kota", "nama_kecamatan"]).sum()
            df_3_temp["tanggal"] = datetime.strptime(temp, '%d-%m-%Y').date()
            df_3 = pd.concat([df_3, df_3_temp], axis=0)

11it [00:00, 916.61it/s]
1it [00:01,  1.30s/it]


In [9]:
#Make the dataframe tidy
df_3 = df_3.loc(axis=0)["JAKARTA TIMUR"].reset_index().sort_values(["nama_kecamatan", "tanggal"]).reset_index(drop=True)

# Combine All Data

In [10]:
#Final Data with adding missing dates
df_final_temp = pd.concat([df, df_2, df_3], axis=0, sort=False)

In [11]:
#Filling missing dates with NaN values
df_final = pd.DataFrame()
for region, df_region in df_final_temp.groupby('nama_kecamatan'):
    temp = df_region.set_index("tanggal").asfreq("D")
    temp[["nama_kecamatan"]] = temp[["nama_kecamatan"]].fillna(region)
    temp["positif"].interpolate(method="akima", inplace=True)
    temp["positif"] = temp["positif"].astype("int")
    df_final = pd.concat([df_final, temp], axis=0)
df_final = df_final.reset_index().sort_values(by=["nama_kecamatan", "tanggal"]).reset_index(drop=True)

## Save the Data!

In [12]:
df_final.to_csv("data_all_region.csv", index=False)